# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [1]:
experiment_name = "Time_Bcast"

In [2]:
%%bash -s
git log -n 1
python --version

commit 66f8d3bb1b6268e8694b09d7cc5502aaa54f9a5a
Author: Najwa Ez Zine <nezzine@frennes>
Date:   Wed Jun 27 11:10:28 2018 +0200

    Journal update


Python 2.7.13


#### Préparation de l'environnement

In [3]:
%%bash -s
pip3 install --user execo
pip3 install --user requests
pip3 install --user termcolor

  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl
  Running setup.py bdist_wheel for termcolor: started
  Running setup.py bdist_wheel for termcolor: finished with status 'done'
  Stored in directory: /home/nezzine/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built termcolor


In [4]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output
import random
from threading import Thread
import math
import collections
from collections import deque
from termcolor import *
from collections import OrderedDict

##### Réservation noeuds

In [5]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 3
# Temps réservation
walltime = "3:0:0" 

# filters out Nantes's econome cluster
resources_selection = "-p \"cluster in ('ecotype','parasilo','grisou','uvb','paravance')\"" 

# Nancy
site = "nancy"

In [6]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1591530


On les range

In [8]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('grisou-7.nancy.grid5000.fr'),
 Host('grisou-8.nancy.grid5000.fr'),
 Host('grisou-9.nancy.grid5000.fr')]

##### Déploiement de l'environnement

###### Via fichier 

In [9]:
force_redeploy = True # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [10]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

nancy: Deployment #D-4e5d75ec-bedb-4dfa-b43f-7b5ff63909c4 started
nancy: Grab the key file /home/nezzine/.ssh/authorized_keys
nancy: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
nancy: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
nancy: Launching a deployment on grisou-[7-9].nancy.grid5000.fr
nancy: Performing a Deploy[SetDeploymentEnvUntrusted] step
nancy:   switch_pxe
nancy:   reboot
nancy:    * Performing a soft reboot on grisou-[7-9].nancy.grid5000.fr
nancy:   wait_reboot
nancy:   send_key_in_deploy_env
nancy:   create_partition_table
nancy:   format_deploy_part
nancy:   mount_deploy_part
nancy:   format_tmp_part
nancy:   format_swap_part
nancy: End of step Deploy[SetDeploymentEnvUntrusted] after 140s
nancy: Performing a Deploy[BroadcastEnvKascade] step
nancy:   send_environment
nancy:    * Broadcast time: 70s
nancy:   manage_admin_post_install
nancy:   manage_user_post_install
nancy:   check_ker

nancy: Warning: Permanently added 'nancy.grid5000.fr,172.16.79.101' (ECDSA) to the list of known hosts.
nancy: Connection to nancy.grid5000.fr closed.


In [11]:
print("Deployement status:\n* ok: {}\n* failed: {}".format(deploy_ok, deploy_failed))

Deployement status:
* ok: {'grisou-7.nancy.grid5000.fr', 'grisou-9.nancy.grid5000.fr', 'grisou-8.nancy.grid5000.fr'}
* failed: set()


##### Avec nom

In [ ]:
deployed, undeployed = deploy(Deployment(nodes, env_name = "debian9-x64-base",other_options="-r ext4 --no-debug-mode"),stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

In [ ]:
print("Deployement status:\n* ok: {}\n* failed: {}".format(deployed, undeployed))

#### Test : commande simple 'ls'

In [ ]:
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

#### Installation environnement/tools sur noeuds

In [12]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [13]:
Remote_install.run().ok

True

#### Version Simgrid

In [14]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [15]:
path_base = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name
path_src = path_base + '/src/'
path_rslt = path_base + '/results/'

#### Récupération fichiers sources

In [ ]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path_src+"griffon.xml",path_src+"griffon_hostfile.txt",path_src+"broadcast.c",path_src+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
Result_get.run().ok

In [ ]:
for i in range(0, nodecount):
    execo.action.Get(hosts = nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location = path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Delayer

In [64]:
def init_tab(num, elt) :
    D = [elt] * num
    return D

In [65]:
# Parsing the results
def parser(num_node,date_precise):
    with open('/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/'
               + date_precise.split('_')[0] + '/'
              + get_host_shortname(nodes[num_node]) + '_'
              + date_precise
              + '.txt') as f:
        return f.readlines()

In [66]:
# Converting and organizing data
def tab_conv(l,NPROCS,precision) :
    S = init_tab(NPROCS, None)
        
    # Taking useful data    
    for i in range(0,NPROCS) :
        line = l[i].split()
        S[int(line[1])] = round(float(line[5]),precision)
    return S

In [67]:
def delayer(num_node,NPROCS,date,precision):
    D = init_tab(NPROCS,0.0)
    
    raw_data = parser(num_node,date)
    
    S = tab_conv(raw_data,NPROCS,precision)
    
    for p in range(0, NPROCS):
        D[p] = round(S[p] - min(S),precision)
    return [D,min(S)]

In [55]:
a = round(13.6958585,2)
b = round(48.1465614,3)


48.147

In [ ]:
# Example
delayer(0,10,'2018-06-04_12:38')

##### Shortcut : séquentiel

In [20]:
def setup() :
    Src_dir = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
    Src_get = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
    Rslt_dir = \
        execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))

# On récup les src
    Src_dir.run().ok
    Src_get.run().ok
    Rslt_dir.run().ok


In [ ]:
# Test : trajectoire jusqu'à la ième machine raccrochée à la trajectoire initiale
def coherence(tab,i):
     return all(x == True for x in tab[0:i])

In [ ]:

def parl_exec () :
    setup()
    # Nbr pcs
    NPROCS = 10
    
    # Nbr itérations
    N = 100
    
    D = init_tab(NPROCS,0.0)
    delay = ' '.join(str(e) for e in D)
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    print(date_precise)
    
    # TODO //
    for i in range (0,nodecount) :
        MIN = (int)((i*N)/len(nodes))
        MAX = (int)(((i+1)*N)/len(nodes))
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(MIN)
             +" MAX = "+str(MAX)
             +" N = "+str(N)
             +" DELAY = '"+ delay +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(MIN) 
                                  +' MAX='+str(MAX)+' N='+str(N)
                                  +" DELAY='"+delay
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
        execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok
        print(delayer(i,NPROCS,date_precise))
    return;

parl_exec()

for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location=path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Parallel simulation

In [21]:
# Thread pour la simulation // opt.
class ParSimer(Thread):

    """Thread chargé simplement d'afficher une lettre dans la console."""
    
    # Paramètrage du thread
    def __init__(self,i,MIN,MAX,N,DELAY):
        Thread.__init__(self)
        self.MIN = MIN
        self.MAX = MAX
        self.N = N
        self.DELAY = DELAY
        self.i = i
    
    # Set la valeur du délai
    def set_delay(self,delay):
        self.DELAY = delay
    
    def get_delay(self):
        return self.DELAY
    def get_MIN(self):
        return self.MIN
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        """print(get_host_shortname(nodes[self.i])+" => "+str(self.i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(self.MIN)
             +" MAX = "+str(self.MAX)
             +" N = "+str(self.N)
             +" DELAY = '"+ self.DELAY +"'\n"
            )
        """
        # make run sur toutes la machine dont le thread s'occupe avec ses paramètres
        execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(self.MIN) 
                                  +' MAX='+str(self.MAX)+' N='+str(self.N)
                                  +" DELAY='"+self.DELAY
                                  +"'",hosts=nodes[self.i],connection_params={'user':'root'}).run().ok


In [22]:
# Formate le tableau en string pour pouvoir etre passé en ligne de commande
def format_cmd(data):
    return (' '.join(str(e) for e in data))

In [23]:
# Formate le string en tableau d'int pour être manipulé 
def unformat_cmd(data):
    return list(map(float,data.split()))

In [69]:
# SOURCE : https://stackoverflow.com/questions/3229419/how-to-pretty-print-nested-dictionaries
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + colored (str(key), "magenta"))
        if isinstance(value, type({})):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

In [58]:
def generate_Dprime(NPROCS,border_inf,border_sup,precision):
    new_Dprime = []
    if (border_inf == 0):
        for p in range(NPROCS):
            new_Dprime.append(round(random.uniform(border_inf,border_sup[p]+1),precision))
    else : 
        for p in range(NPROCS):
            new_Dprime.append(round(random.choice(range(border_inf[p]+1,border_sup[p]+1)),precision))
    return format_cmd(new_Dprime)

In [ ]:
bold
dark
underline
blink
reverse
concealed

In [26]:
import sys
from termcolor import colored, cprint

text = colored('Hello, World!', 'red', attrs=[ 'blink'])
print(text)
text = colored('Hello, World!', 'red', attrs=['dark'])
print(text)
text = colored('Hello, World!', 'red', attrs=[ 'underline'])
print(text)
text = colored('Hello, World!', 'red', attrs=[ 'bold'])
print(text)
text = colored('Hello, World!', 'red', attrs=[ 'concealed'])
print(text)
cprint('Hello, World!', 'green', 'on_red',attrs=[ 'reverse'])

Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!


In [61]:
# SIMULATION
def simulation(N, NPROCS, fragment_size, precision, mode, catch_me):
    """Contrôle de la validité des paramètres"""
    # Taille du fragment ne peut être supérieure à la zone d'itération
    if (fragment_size > N): 
        raise ValueError("Fragment size cannot be superior to N")
        
    """Mise en place de l'environnement et des variables"""
    # Environnement
    setup() 
    
    # Dates: pour la gestion de fichiers (IMPORTANT pour lecture/écriture)
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    date = date_precise.split('_')[0]
    
    # Total execution time
    total_execution_time = 0
    
    # Tableau de threads
    machines = init_tab(nodecount, None)
      
    # position trajectoire cohérente courante
    t_curr = 0
    
    # D de la trajectorie cohérente courante
    D_curr = format_cmd(init_tab(NPROCS, 0))
    
    # Taille du dernier slot
    last_fragment_size = N % fragment_size
    
    # Nombre de fragments
    fragment_number = int((math.ceil(N / fragment_size)))
    
    # Nombre de rounds
    rounds_taken = 0
    
    # zone d'itérations à explorer
    To_explore = collections.deque()
    
    # Initialisation des zones à explorer D = '0...0' pour chaque zone T -> (Min d'itération , D initial)
    for i in range(fragment_number):
        To_explore.append((i * fragment_size, format_cmd(init_tab(NPROCS, 0))))

    # Round actuel
    curr_round = 0
    
    # Liste des trajectoires pour un temps t et un D donnée
    sim = OrderedDict()
    



    success = False 
    
    catch_me_ind = 0
    
    while(True):
        curr_round = curr_round + 1
        
        # Trajectoire cohérente complète trouvé : FIN DE SIMULATION
        if (t_curr >= N) :
            cprint ("Total time : " + str(total_execution_time) , "grey" , "on_yellow")
            cprint ("Success : " + str(success) , "grey" , "on_green")
            break;        
        if (len(To_explore) == 0):
            raise 
        # Il n'existe pas encore de trajectoire cohérente -> Nouveau round
        cprint("--------------------------------------------  ROUND :  " + str(curr_round) + "   --------------------------------------------\n","red")
        
        # Zones restantes à explorer
        number_paths_To_explore = len(To_explore)
        
        # Nombre de tâches parallèles lançables
        number_launchable_tasks = min(nodecount, number_paths_To_explore)
        
        # Affichage des paramètres
        cprint ("  || LAUNCHING || \n","green")
        print ( colored("TO EXPLORE : " , "blue"), str(To_explore)) 
        
        """ Paramètrage des machines """
        
        # Soit j'ai plus de zone à explorer que de machines disponibles soit l'inverse
        for q in range(number_launchable_tasks):
            
            # On récupère la première zone à explorer
            parameters = To_explore.popleft()
            
            # MIN est le champ de gauche
            MIN = parameters[0]
            
            # MAX de la zone à explorer vaut le min + la taille du slot
            MAX = min (MIN + fragment_size, N)
            
            # D est le champ de droite
            delay = parameters[1]
            
            # Paramètrage des machines
            machines[q] = ParSimer(q, MIN, MAX, N, delay)
            #print("Machine " + str(q) + " -> MIN :" + str(MIN) + " MAX : " + str(MAX) + " D : "+ delay)   
                
        """ Simulation """  
        # Lancement de la simulation
        for q in range(number_launchable_tasks):
            machines[q].start()
            
        # Fin de simulation
        for q in range(number_launchable_tasks) :
            machines[q].join()
            # Récupération des résultats
            execo.action.Get(hosts=nodes[q],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[q]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok    
            
        """ Analyse des résultats"""    
        # Traitement des données
        for q in range(number_launchable_tasks):
            # On récupère les paramètres des machines
            t = machines[q].get_MIN()
            D = machines[q].get_delay()
            
            # Calcul du délai
            # On récupère D et S
            raw_data = delayer(q, NPROCS, date_precise,precision)
            
            # Filtre D : Calcul du délai
            Dprime = format_cmd(raw_data[0])
            
            #Filte S : Calcul du temps d'éxécution total 
            T = raw_data[1]
            
            # Ajout dans ma table (Dprime, round, machine qui l'a calculé, temps)
            sim.setdefault(t,{}).update({D:(Dprime,curr_round,q,T)})
            MAX = min (t + fragment_size, N)
        
        # Affichage des paramètres
        print ( colored ("T_curr : " , "yellow") , str(t_curr))
        print ( colored ("D_curr : " , "yellow") , str(D_curr))
        print ( colored ("Dprime : ", "yellow") , str(Dprime))
        print ( colored ("total execution time : ", "red") , str(total_execution_time) + "\n") 
        
        merged_paths = 0
        # Calcul de la trajectoire cohérente
        while (t_curr < N and t_curr in sim and D_curr in sim[t_curr]):
            merged_paths += 1
            # Update le temps d'exécution
            total_execution_time += sim[t_curr][D_curr][3]
            
            # Update du temps d'exécution dans le dernier fragment
            if (t_curr + fragment_size >= N) : total_execution_time += max(unformat_cmd(sim[t_curr][D_curr][0]))
            
            # Update des t_curr et D_curr
            D_curr = sim[t_curr][D_curr][0]
            t_curr = min(t_curr + fragment_size, N)
        if (merged_paths > 1) : 
            cprint("****** SUCCESS : " + str(merged_paths), attrs=['reverse', 'bold'])
            success = True 
        
        # Affichage des résultats
        cprint ("\n  || RESULTS || \n","red")
        print ( colored ( "TO EXPLORE : " , "blue"), str(To_explore)) 
        pretty (sim) 
        print ( colored ("T_curr : " , "yellow"), str(t_curr))
        print ( colored ("D_curr : " , "yellow") , str(D_curr))
        print ( colored ("Dprime : ", "yellow") , str(Dprime))
        print ( colored ("total execution time : ", "red") , str(total_execution_time) + "\n") 
        
        
        """ Nouveau paramètrages """
        cprint("  || ANALYSIS || \n","magenta")
        
        for q in range(number_launchable_tasks):
            t = machines[q].get_MIN()
            D = machines[q].get_delay()
            
            MAX = min(t + fragment_size, N)
            Dprime = sim[t][D][0]
           
            # Zone valide
            if(MAX < N):
                # Zone pas encore relié à la trajectoire initiale
                if (MAX > t_curr):
                    caught = False
                # Ajout dans les zones à explorer du nouveau point de départ à exploiter en fonction du mode
                    # Test catch me sur 2^catch_me_ind round
                    if (catch_me and 1 << catch_me_ind == curr_round):
                        catch_me_ind += 1
                        try :
                            caught = Dprime in sim[MAX]
                        except KeyError:
                            pass 
                       
                    if (not catch_me or not caught):
                        borders = unformat_cmd(Dprime)
                        double_Dprime = [(2*(x+1)) for x in borders]
                      
                        # Bornes pour le random 
                        border_inf = 0
                        border_sup = double_Dprime
                        if (mode == "RAND_SUP") : 
                            border_sup = borders
                            Dprime =  generate_Dprime(NPROCS, border_inf, border_sup,precision)
                        elif (mode == "RAND_INF") : 
                            border_inf = borders
                            Dprime = generate_Dprime(NPROCS, border_inf, border_sup,precision)
                        elif (mode == "RAND") : 
                            Dprime =  generate_Dprime(NPROCS, border_inf, border_sup,precision)
                        print("["+str(MAX) +"] Generated " + str(mode) + " Dprime : " + str(Dprime))
                To_explore.appendleft((MAX, Dprime))
        print(colored ( "TO EXPLORE : " , "blue") , str(To_explore) + "\n")
        

In [83]:
# Nbr pcs
NPROCS = 5
    
# Nbr itérations
N = 10000

# taille paquet d'itérations
fragment_size = 2500
    
# precision
precision = 3

# mode de génération de l'état initial à chaque round
mode_opt = ["FIX","RAND","RAND_INF","RAND_SUP"]
mode = mode_opt[1]

catch_me = False

# Lancement de la simulation
simulation(N,NPROCS,fragment_size,precision,mode,catch_me)

--------------------------------------------  ROUND :  1   --------------------------------------------

  || LAUNCHING || 

TO EXPLORE :  deque([(0, '0 0 0 0 0'), (2500, '0 0 0 0 0'), (5000, '0 0 0 0 0'), (7500, '0 0 0 0 0')])
T_curr :  0
D_curr :  0 0 0 0 0
Dprime :  0.001 0.001 0.001 0.0 0.0
total execution time :  0


  || RESULTS || 

TO EXPLORE :  deque([(7500, '0 0 0 0 0')])
0
	0 0 0 0 0
		('0.001 0.001 0.001 0.0 0.0', 1, 0, 0.483)
2500
	0 0 0 0 0
		('0.001 0.001 0.001 0.0 0.0', 1, 1, 0.483)
5000
	0 0 0 0 0
		('0.001 0.001 0.001 0.0 0.0', 1, 2, 0.483)
T_curr :  2500
D_curr :  0.001 0.001 0.001 0.0 0.0
Dprime :  0.001 0.001 0.001 0.0 0.0
total execution time :  0.483

  || ANALYSIS || 

[5000] Generated RAND Dprime : 1.164 2.545 1.39 0.055 2.039
[7500] Generated RAND Dprime : 2.755 1.103 2.274 1.839 0.654
TO EXPLORE :  deque([(7500, '2.755 1.103 2.274 1.839 0.654'), (5000, '1.164 2.545 1.39 0.055 2.039'), (2500, '0.001 0.001 0.001 0.0 0.0'), (7500, '0 0 0 0 0')])

---------------

#### Publication des résultats

In [ ]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'---'"$2"" Float"

#### Fin d'expérience : suppression du job

In [ ]:
oardel(jobid)